In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt
import seaborn as sn
import random
import tensorflow as tf
import os
import pickle
from zipfile import ZipFile
from tqdm import tqdm
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
pd.set_option('display.max_columns' ,None)

In [4]:
DIR='../input/recruit-restaurant-visitor-forecasting'

In [5]:
for curr,folders,files in os.walk(DIR):
    for file in files:
        if file!="recruit-restaurant-visitor-forecasting.zip":
            if file.endswith(".zip"):
                f=ZipFile(os.path.join(curr,file))
                f.extractall()
                f.close()

In [6]:
air_res=pd.read_csv("air_reserve.csv")
air_info=pd.read_csv("air_store_info.csv")
##########################################
hpg_res=pd.read_csv('hpg_reserve.csv')
hpg_info=pd.read_csv('hpg_store_info.csv')
##########################################
date_info=pd.read_csv("date_info.csv")
stores_map=pd.read_csv('store_id_relation.csv')
###########################################
test=pd.read_csv("sample_submission.csv")
##################################
air_visit=pd.read_csv("air_visit_data.csv")

In [7]:
test['air_store_id']=test['id'].apply(lambda x: "_".join(x.split("_")[:-1]))
test['visit_date']=test['id'].apply(lambda x: x.split("_")[-1])

restaurants which are present in both websites

In [8]:
stores_map.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


table contains restaurant id, visit date(when the customer vists), reserve date(when the reservation is made) and number of customers booked

In [9]:
air_res.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [10]:
hpg_res.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [11]:
hpg_res=pd.merge(left=hpg_res,right=stores_map,left_on="hpg_store_id",right_on='hpg_store_id',how='inner')

In [12]:
hpg_res.drop("hpg_store_id",axis=1,inplace=True)
reservations=pd.concat([air_res,hpg_res],ignore_index=True)

In [13]:
reservations.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [14]:
def make_datetime_to_date(dataframe):
    dataframe['visit_datetime']=pd.to_datetime(dataframe['visit_datetime'])
    dataframe['reserve_datetime']=pd.to_datetime(dataframe['reserve_datetime'])
    dataframe['n_days_gap']=dataframe.apply(lambda x: (x['visit_datetime']-x['reserve_datetime']).days,axis=1)
    dataframe["n_hrs_gap"]=dataframe.apply(lambda x: (x['visit_datetime']-x['reserve_datetime']).total_seconds()/3600.0,axis=1)
    dataframe["visit_date"]=dataframe['visit_datetime'].dt.date
    dataframe.drop(['visit_datetime',"reserve_datetime"],axis=1,inplace=True)
    return dataframe

In [15]:
reservations=make_datetime_to_date(reservations)

In [16]:
reservations.head()

,air_store_id,reserve_visitors,n_days_gap,n_hrs_gap,visit_date
0,air_877f79706adbfb06,1,0,3.0,2016-01-01
1,air_db4b38ebe7a7ceff,3,0,0.0,2016-01-01
2,air_db4b38ebe7a7ceff,6,0,0.0,2016-01-01
3,air_877f79706adbfb06,2,0,4.0,2016-01-01
4,air_db80363d35f10926,5,0,19.0,2016-01-01


info about the cuisines and near location of the restaurants

In [17]:
air_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [18]:
hpg_info.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [19]:
hpg_info=pd.merge(left=hpg_info,right=stores_map,left_on="hpg_store_id",right_on='hpg_store_id',how='inner')

In [20]:
hpg_info.rename(columns={"hpg_genre_name":"air_genre_name","hpg_area_name":"air_area_name"},inplace=True)
hpg_info.drop("hpg_store_id",axis=1,inplace=True)

In [21]:
information=pd.concat([air_info,hpg_info],ignore_index=True)

calender

In [22]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [23]:
reservations.duplicated(subset=["air_store_id","visit_date"]).unique()

array([False,  True])

In [24]:
information.duplicated(subset=["air_store_id"]).unique()

array([False,  True])

In [25]:
new_cols=["air_store_id","visit_date"]
for col in ['reserve_visitors','n_hrs_gap','n_days_gap']:
    for agg in ['mean','max','min','sum']:
        new_cols.append(col+"_"+agg)

In [26]:
reservations=reservations.groupby(["air_store_id","visit_date"]).agg({"reserve_visitors":["mean","max","min","sum"],
                                                        "n_hrs_gap":["mean","max","min","sum"],
                                                        "n_days_gap":["mean","max","min","sum"]}).reset_index()

In [27]:
reservations.columns=new_cols

In [28]:
reservations['visit_date']=pd.to_datetime(reservations['visit_date'])
reservations.sort_values(by=['air_store_id','visit_date'],ignore_index=True,inplace=True)

In [29]:
cols=['reserve_visitors_sum',"n_hrs_gap_sum","n_days_gap_sum"]
for col in cols:
    for window in [7,14,30]:
        reservations[f"{col}_rolling_{window}_mean"]=reservations.groupby("air_store_id")[col].transform(lambda x: x.rolling(window).mean())
        reservations[f"{col}_rolling_{window}_std"] =reservations.groupby("air_store_id")[col].transform(lambda x: x.rolling(window).std())
        reservations[f"{col}_rolling_{window}_max"] =reservations.groupby("air_store_id")[col].transform(lambda x: x.rolling(window).max())
        reservations[f"{col}_rolling_{window}_min"] =reservations.groupby("air_store_id")[col].transform(lambda x: x.rolling(window).min())

In [30]:
reservations.sort_values(by=['air_store_id','visit_date'],ignore_index=True,inplace=True)

In [31]:
cols=['reserve_visitors_sum',"n_hrs_gap_sum","n_days_gap_sum"]
for col in cols:
    for lag in range(1,8):
        reservations[f"{col}_lag_{lag}"]=reservations.groupby("air_store_id")[col].transform(lambda x: x.shift(lag))        

In [32]:
reservations[['reserve_visitors_sum_rolling_7_mean',
             'reserve_visitors_sum_rolling_14_mean',
             'reserve_visitors_sum_lag_1','reserve_visitors_sum_lag_7',
             'reserve_visitors_sum']].head(15)

,reserve_visitors_sum_rolling_7_mean,reserve_visitors_sum_rolling_14_mean,reserve_visitors_sum_lag_1,reserve_visitors_sum_lag_7,reserve_visitors_sum
0,NaN,NaN,NaN,NaN,2
1,NaN,NaN,2.0,NaN,4
2,NaN,NaN,4.0,NaN,2
3,NaN,NaN,2.0,NaN,2
4,NaN,NaN,2.0,NaN,5
5,NaN,NaN,5.0,NaN,2
6,2.857143,NaN,2.0,NaN,3
7,2.714286,NaN,3.0,2.0,1
8,2.428571,NaN,1.0,4.0,2
9,3.142857,NaN,2.0,2.0,7


In [33]:
information=information.groupby("air_store_id").agg({"air_genre_name":"unique","air_area_name":"unique"}).reset_index()

In [34]:
def get_cuisines(genres):
    l=[]
    for g in genres:
        gs=g.split("/")
        l.extend(gs)
    return list(set(l))

In [35]:
information['n_areas']=information['air_area_name'].apply(lambda x: len(x))

In [36]:
information['n_areas'].unique()

array([1, 2])

# some stores are present at two places


In [37]:
AREA_SPLIT=False

In [38]:
if AREA_SPLIT:
    information['first_area'] =information['air_area_name'].apply(lambda x: x[0])
    information['second_area']=information['air_area_name'].apply(lambda x: x[1] if len(x)>1 else "only at one area")
else:
    information['air_area_name']=information['air_area_name'].apply(lambda x: ",".join(x))

In [39]:
information['air_genre_name']=information['air_genre_name'].apply(get_cuisines)

In [40]:
information.head()

,air_store_id,air_genre_name,air_area_name,n_areas
0,air_00a91d42b08b08d9,"[Italian, French]",Tōkyō-to Chiyoda-ku Kudanminami,1
1,air_0164b9927d20bcc3,"[Italian, French]",Tōkyō-to Minato-ku Shibakōen,1
2,air_0241aa3964b7f861,[Izakaya],Tōkyō-to Taitō-ku Higashiueno,1
3,air_0328696196e46f18,[Dining bar],Ōsaka-fu Ōsaka-shi Nakanochō,1
4,air_034a3d5b40d5b1b1,"[Cafe, Sweets]",Ōsaka-fu Ōsaka-shi Ōhiraki,1


In [41]:
if AREA_SPLIT:
    information.drop(['air_area_name'],axis=1,inplace=True)

In [42]:
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder,StandardScaler

In [43]:
if AREA_SPLIT:
    areas=information['first_area'].unique().tolist()
    areas.extend(information['second_area'].unique().tolist())
else:
    areas=information['air_area_name'].unique().tolist()

In [44]:
area_encoder=LabelEncoder()
area_encoder.fit(areas)
if AREA_SPLIT:
    information['first_area']=area_encoder.transform(information['first_area'])
    information['second_area']=area_encoder.transform(information['second_area'])
else:
    information['air_area_name']=area_encoder.transform(information['air_area_name'])
##########################################
if AREA_SPLIT:
    cuisine_encoder=MultiLabelBinarizer()
    cuisine_encoder.fit(information['air_genre_name'])
    temp=pd.DataFrame(cuisine_encoder.transform(information['air_genre_name']),columns=list(cuisine_encoder.classes_))
    information=information.join(temp)
    information.drop("air_genre_name",axis=1,inplace=True)
else:
    cuisine_encoder=LabelEncoder()
    information['air_genre_name']=information['air_genre_name'].apply(lambda x: ",".join(x))
    information['air_genre_name']=cuisine_encoder.fit_transform(information['air_genre_name'])

In [45]:
if AREA_SPLIT:
    information['Asian'].value_counts()
    information.loc[information['Asian']==1,"first_area"].unique(),information.loc[information['Asian']==1,"second_area"].unique()

In [46]:
if AREA_SPLIT:
    for c in list(cuisine_encoder.classes_):
        sec={c:information.groupby('second_area')[c].transform("sum").values,"sec":information['second_area'].values}
        sec=pd.DataFrame(sec)
        sec.loc[sec['sec']==119,c]=0
        information[f"n_rest_of_{c}_per_area"]=information.groupby('first_area')[c].transform("sum")+sec[c]
else:
    information['count_res_per_genre_area']=information.groupby(['air_area_name','air_genre_name'])['air_store_id'].transform("count")
    information['count_genres_per_area']=information.groupby(['air_area_name'])['air_genre_name'].transform("count")

In [47]:
if AREA_SPLIT:
    sec=pd.DataFrame({"n_res":information.groupby("second_area")['air_store_id'].transform("count").values,
                  "sec":information['second_area'].values})
    sec.loc[sec['sec']==119,'n_res']=0
    information["n_restaurants_per_area"]=information.groupby("first_area")['air_store_id'].transform("count")+sec['n_res']

In [48]:
information.head()

,air_store_id,air_genre_name,air_area_name,n_areas,count_res_per_genre_area,count_genres_per_area
0,air_00a91d42b08b08d9,18,58,1,4,19
1,air_0164b9927d20bcc3,18,82,1,13,51
2,air_0241aa3964b7f861,19,108,1,3,15
3,air_0328696196e46f18,9,126,1,2,2
4,air_034a3d5b40d5b1b1,4,134,1,2,4


In [49]:
date_info['calendar_date']=pd.to_datetime(date_info['calendar_date'])

In [50]:
date_info.groupby('day_of_week')['holiday_flg'].sum()

day_of_week
Friday       5
Monday       9
Saturday     5
Sunday       4
Thursday     7
Tuesday      2
Wednesday    3
Name: holiday_flg, dtype: int64

In [51]:
date_info['day_of_week'].value_counts()

Friday       74
Saturday     74
Sunday       74
Monday       74
Tuesday      74
Wednesday    74
Thursday     73
Name: day_of_week, dtype: int64

In [52]:
date_info.loc[date_info['day_of_week'].isin(['Saturday','Sunday']),"holiday_flg"]=1

made sunday, saturdays as holidays

In [53]:
date_info.groupby('day_of_week')['holiday_flg'].sum()

day_of_week
Friday        5
Monday        9
Saturday     74
Sunday       74
Thursday      7
Tuesday       2
Wednesday     3
Name: holiday_flg, dtype: int64

In [54]:
date_info['month']=date_info['calendar_date'].dt.month
date_info['day']=date_info['calendar_date'].dt.day
for i in date_info.loc[((date_info['month']==4) & (date_info['day']>=29)) | ((date_info['month']==5) & (date_info['day']<=7))].index:
    day=date_info.loc[i,'day']
    if day not in [6,7]:
        date_info.loc[i,'holiday_flg']=1
    else:
        if day==6:
            if date_info.loc[i+1,'day']==7 and date_info.loc[i+1,'holiday_flg']==1:
                date_info.loc[i,'holiday_flg']=1

In [55]:
q='''
with f_cte as
(
select a.calendar_date,b.calendar_date as sec,b.holiday_flg,a.day_of_week,a.month,a.day
from date_info a
inner join
date_info b
on b.calendar_date>=a.calendar_date and (a.holiday_flg=1 and b.holiday_flg=1)
),final_cte as
(
select *,row_number() over(partition by calendar_date order by sec) as row_id
from
(
select *,julianday(sec)-julianday(calendar_date) as diff
from f_cte
) a
)
select *,sum(holiday_flg) as n_holidays
from final_cte
where row_id-diff=1
group by calendar_date


'''
a=ps.sqldf(q)
a.loc[((a['month']==4) & (a['day']>=29)) | ((a['month']==5) & (a['day']<=7))]

,calendar_date,sec,holiday_flg,day_of_week,month,day,diff,row_id,n_holidays
38,2016-04-29 00:00:00.000000,2016-04-29 00:00:00.000000,1,Friday,4,29,0.0,1,10
39,2016-04-30 00:00:00.000000,2016-04-30 00:00:00.000000,1,Saturday,4,30,0.0,1,9
40,2016-05-01 00:00:00.000000,2016-05-01 00:00:00.000000,1,Sunday,5,1,0.0,1,8
41,2016-05-02 00:00:00.000000,2016-05-02 00:00:00.000000,1,Monday,5,2,0.0,1,7
42,2016-05-03 00:00:00.000000,2016-05-03 00:00:00.000000,1,Tuesday,5,3,0.0,1,6
43,2016-05-04 00:00:00.000000,2016-05-04 00:00:00.000000,1,Wednesday,5,4,0.0,1,5
44,2016-05-05 00:00:00.000000,2016-05-05 00:00:00.000000,1,Thursday,5,5,0.0,1,4
45,2016-05-06 00:00:00.000000,2016-05-06 00:00:00.000000,1,Friday,5,6,0.0,1,3
46,2016-05-07 00:00:00.000000,2016-05-07 00:00:00.000000,1,Saturday,5,7,0.0,1,2
163,2017-04-29 00:00:00.000000,2017-04-29 00:00:00.000000,1,Saturday,4,29,0.0,1,9


In [56]:
a['calendar_date']=pd.to_datetime(a['calendar_date'])
a.drop(['sec','holiday_flg','day_of_week','month','day','diff','row_id'],axis=1,inplace=True)
a.head()

,calendar_date,n_holidays
0,2016-01-01,3
1,2016-01-02,2
2,2016-01-03,1
3,2016-01-09,3
4,2016-01-10,2


In [57]:
date_info=pd.merge(left=date_info,right=a,left_on='calendar_date',right_on='calendar_date',how='left')
date_info['n_holidays'].fillna(0,inplace=True)
date_info.drop(['day_of_week','month','day'],axis=1,inplace=True)

In [58]:
q='''
with f_cte as
(
select *,(select min(b.calendar_date)
          from date_info b
          where b.calendar_date>=a.calendar_date and b.holiday_flg!=a.holiday_flg) as max_date
from date_info a
),final_cte as
(
select a.calendar_date,a.holiday_flg,a.n_holidays,a.max_date,b.calendar_date as temp_date
from f_cte a
left join
f_cte b
on a.max_date=b.max_date
),last_cte as
(
select calendar_date,holiday_flg,n_holidays,max_date,julianday(max_date)-julianday(min(temp_date)) as temp_vacation_length
from final_cte
group by calendar_date,holiday_flg,n_holidays,max_date
)
select calendar_date,holiday_flg,n_holidays,
       (case when holiday_flg=1 then temp_vacation_length
       else 0
       end) as vacation_length
from last_cte
'''
date_info=ps.sqldf(q)

In [59]:
date_info['calendar_date']=pd.to_datetime(date_info['calendar_date'])
date_info['month']=date_info['calendar_date'].dt.month
date_info['day']=date_info['calendar_date'].dt.day
date_info.loc[((date_info['month']==4) & (date_info['day']>=29)) | ((date_info['month']==5) & (date_info['day']<=7))]

,calendar_date,holiday_flg,n_holidays,vacation_length,month,day
119,2016-04-29,1,10.0,10.0,4,29
120,2016-04-30,1,9.0,10.0,4,30
121,2016-05-01,1,8.0,10.0,5,1
122,2016-05-02,1,7.0,10.0,5,2
123,2016-05-03,1,6.0,10.0,5,3
124,2016-05-04,1,5.0,10.0,5,4
125,2016-05-05,1,4.0,10.0,5,5
126,2016-05-06,1,3.0,10.0,5,6
127,2016-05-07,1,2.0,10.0,5,7
484,2017-04-29,1,9.0,9.0,4,29


In [60]:
date_info.drop(['month','day'],axis=1,inplace=True)

In [61]:
test['visit_date']=pd.to_datetime(test['visit_date'])

In [62]:
air_visit['visit_date']=pd.to_datetime(air_visit['visit_date'])

In [63]:
air_visit=pd.merge(left=air_visit,right=date_info,left_on="visit_date",right_on='calendar_date',how='left')
air_visit.drop(['calendar_date'],axis=1,inplace=True)
###########################################
test=pd.merge(left=test,right=date_info,left_on="visit_date",right_on='calendar_date',how='left')
test.drop(['calendar_date'],axis=1,inplace=True)

In [64]:
air_visit['visit_month']=air_visit['visit_date'].dt.month
air_visit['visit_dow']=air_visit['visit_date'].dt.dayofweek
air_visit['visit_day']=air_visit['visit_date'].dt.day
########################################
test['visit_month']=test['visit_date'].dt.month
test['visit_dow']=test['visit_date'].dt.dayofweek
test['visit_day']=test['visit_date'].dt.day

In [65]:
air_visit=pd.merge(left=air_visit,right=reservations,left_on=['air_store_id','visit_date'],right_on=['air_store_id','visit_date'],how='left')
air_visit=pd.merge(left=air_visit,right=information,left_on='air_store_id',right_on='air_store_id',how='left')
##########################
test=pd.merge(left=test,right=reservations,left_on=['air_store_id','visit_date'],right_on=['air_store_id','visit_date'],how='left')
test=pd.merge(left=test,right=information,left_on='air_store_id',right_on='air_store_id',how='left')

In [66]:
test.head()

,id,visitors,air_store_id,visit_date,holiday_flg,n_holidays,vacation_length,visit_month,visit_dow,visit_day,reserve_visitors_mean,reserve_visitors_max,reserve_visitors_min,reserve_visitors_sum,n_hrs_gap_mean,n_hrs_gap_max,n_hrs_gap_min,n_hrs_gap_sum,n_days_gap_mean,n_days_gap_max,n_days_gap_min,n_days_gap_sum,reserve_visitors_sum_rolling_7_mean,reserve_visitors_sum_rolling_7_std,reserve_visitors_sum_rolling_7_max,reserve_visitors_sum_rolling_7_min,reserve_visitors_sum_rolling_14_mean,reserve_visitors_sum_rolling_14_std,reserve_visitors_sum_rolling_14_max,reserve_visitors_sum_rolling_14_min,reserve_visitors_sum_rolling_30_mean,reserve_visitors_sum_rolling_30_std,reserve_visitors_sum_rolling_30_max,reserve_visitors_sum_rolling_30_min,n_hrs_gap_sum_rolling_7_mean,n_hrs_gap_sum_rolling_7_std,n_hrs_gap_sum_rolling_7_max,n_hrs_gap_sum_rolling_7_min,n_hrs_gap_sum_rolling_14_mean,n_hrs_gap_sum_rolling_14_std,n_hrs_gap_sum_rolling_14_max,n_hrs_gap_sum_rolling_14_min,n_hrs_gap_sum_rolling_30_mean,n_hrs_gap_sum_rolling_30_std,n_hrs_gap_sum_rolling_30_max,n_hrs_gap_sum_rolling_30_min,n_days_gap_sum_rolling_7_mean,n_days_gap_sum_rolling_7_std,n_days_gap_sum_rolling_7_max,n_days_gap_sum_rolling_7_min,n_days_gap_sum_rolling_14_mean,n_days_gap_sum_rolling_14_std,n_days_gap_sum_rolling_14_max,n_days_gap_sum_rolling_14_min,n_days_gap_sum_rolling_30_mean,n_days_gap_sum_rolling_30_std,n_days_gap_sum_rolling_30_max,n_days_gap_sum_rolling_30_min,reserve_visitors_sum_lag_1,reserve_visitors_sum_lag_2,reserve_visitors_sum_lag_3,reserve_visitors_sum_lag_4,reserve_visitors_sum_lag_5,reserve_visitors_sum_lag_6,reserve_visitors_sum_lag_7,n_hrs_gap_sum_lag_1,n_hrs_gap_sum_lag_2,n_hrs_gap_sum_lag_3,n_hrs_gap_sum_lag_4,n_hrs_gap_sum_lag_5,n_hrs_gap_sum_lag_6,n_hrs_gap_sum_lag_7,n_days_gap_sum_lag_1,n_days_gap_sum_lag_2,n_days_gap_sum_lag_3,n_days_gap_sum_lag_4,n_days_gap_sum_lag_5,n_days_gap_sum_lag_6,n_days_gap_sum_lag_7,air_genre_name,air_area_name,n_areas,count_res_per_genre_area,count_genres_per_area
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,1,1.0,2.0,4,6,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,58,1,4,19
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,0,0.0,0.0,4,0,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,58,1,4,19
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,0,0.0,0.0,4,1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,58,1,4,19
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,0,0.0,0.0,4,2,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,58,1,4,19
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,0,0.0,0.0,4,3,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,58,1,4,19


In [67]:
air_visit.sort_values(by=['air_store_id','visit_date'],ignore_index=True,inplace=True)
########################
test.sort_values(by=['air_store_id','visit_date'],ignore_index=True,inplace=True)

In [68]:
cols=["visitors"]
for col in cols:
    for lag in range(1,8):
        air_visit[f"{col}_lag_{lag}"]=air_visit.groupby("air_store_id")[col].transform(lambda x: x.shift(lag))  

In [69]:
test.fillna(-1,inplace=True)
air_visit.fillna(-1,inplace=True)

In [70]:
restaurant_encoder=LabelEncoder()
air_visit['air_store_id']=restaurant_encoder.fit_transform(air_visit['air_store_id'])
test['air_store_id']=restaurant_encoder.transform(test['air_store_id'])

In [71]:
with open(f"../input/restaurants-visiters/predictive_features,.pkl","rb") as f:
    PREDICTIVE_FEATURES=pickle.load(f)

In [72]:
air_visit=air_visit.loc[air_visit['air_store_id'].isin(test['air_store_id'].unique())].reset_index(drop=True)

In [73]:
air_visit.sort_values(by=['air_store_id','visit_date'],ignore_index=True,inplace=True)
test.sort_values(by=['air_store_id','visit_date'],ignore_index=True,inplace=True)

In [74]:
#import xgboost as xgb
#with open("../input/restaurants-visiters/xgboost_params.pkl","rb") as f:
#    params=pickle.load(f)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Embedding,BatchNormalization,Dense,ReLU,Dropout

In [75]:
def Embedding_out_dim(n_cat):
    return min(600, round(1.6 * n_cat ** .56))

In [76]:
categorical_columns=['air_store_id','visit_month','visit_dow','visit_day','air_genre_name','air_area_name']
embedding_dims={'air_store_id': {'input_dim': 829, 'out_dim': 69},
 'visit_month': {'input_dim': 13, 'out_dim': 7},
 'visit_dow': {'input_dim': 7, 'out_dim': 5},
 'visit_day': {'input_dim': 32, 'out_dim': 11},
 'air_genre_name': {'input_dim': 36, 'out_dim': 12},
 'air_area_name': {'input_dim': 135, 'out_dim': 25}}

In [77]:
numerical_columns=list(col for col in PREDICTIVE_FEATURES if col not in categorical_columns)

In [78]:
def RMSLE(y_true,y_pred):
    y_true=tf.cast(y_true,dtype=y_pred.dtype)
    log_1p_y_true=tf.keras.backend.log(1.0+y_true)
    log_1p_y_pred=tf.keras.backend.log(1.0+y_pred)
    sq_diff=tf.reduce_mean(tf.keras.backend.square(log_1p_y_true-log_1p_y_pred),axis=-1)
    mean_value=tf.reduce_mean(sq_diff,axis=-1)
    return mean_value**0.5

In [79]:
def Make_LSTM_Model(parameters):
    window_size=parameters['window_size']
    lstm_n_layers=parameters['lstm_n_layers']
    dense_layers=parameters['dense_layers']
    lstm_units=parameters['lstm_units']
    dense_units=parameters['dense_units']
    ins_dict={}
    embed_layers={}
    ins_dict['numeric']=Input((window_size,len(numerical_columns)),name='numerical')
    for col,dims in embedding_dims.items():
        ins_dict[col]=Input((window_size,),name=f"ins_{col}")
        embed_layers[col]=Embedding(input_dim=dims['input_dim'],output_dim=dims['out_dim'],
                                    input_length=window_size,name=f"embedding_{col}")(ins_dict[col])
    concat_layer=tf.keras.layers.Concatenate()
    l=[ins_dict['numeric']]
    for col,layer in embed_layers.items():
        l.append(layer)
    x=concat_layer(l)
    for layer in range(lstm_n_layers):
        if layer!=lstm_n_layers-1:
            x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units[layer],return_sequences=True))(x)
            #x=tf.keras.layers.SpatialDropout1D(0.1)(x)
        else:
            x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units[layer]))(x)
    #x=tf.keras.layers.Dropout(0.1)(x)
    if dense_layers>0:
        for layer in range(dense_layers):
            x=Dense(dense_units[layer])(x)
            x=BatchNormalization()(x)
            x=ReLU()(x)
            #x=tf.keras.layers.Dropout(0.1)(x)
    outs=Dense(1,activation="relu")(x)
    model=Model(inputs=ins_dict,outputs=outs)
    return model

In [80]:
parameters={"window_size":19,"lstm_n_layers":2,"lstm_units":[30,30],"dense_layers":1,"dense_units":[30]}

In [81]:
model=Make_LSTM_Model(parameters)
model.summary()

2022-07-12 09:25:21.621328: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ins_air_store_id (InputLayer)   [(None, 19)]         0                                            
__________________________________________________________________________________________________
ins_visit_month (InputLayer)    [(None, 19)]         0                                            
__________________________________________________________________________________________________
ins_visit_dow (InputLayer)      [(None, 19)]         0                                            
__________________________________________________________________________________________________
ins_visit_day (InputLayer)      [(None, 19)]         0                                            
______________________________________________________________________________________________

In [82]:
model.load_weights("../input/restaurants-visiters/lstm_2_dense_1.h5")

In [83]:
def MAKE_LAG_FEATURES(data):
    for lag in range(1,8):
        data[f"{col}_lag_{lag}"]=data.groupby("air_store_id")["visitors"].transform(lambda x: x.shift(lag))  
    return data
def MAKE_ROLLING_FEATURES(data):
    col='visitors'
    for window in [7,14,30]:
        data[f"{col}_rolling_{window}_mean"]=data.groupby("air_store_id")[col].transform(lambda x: x.rolling(window).mean())
        data[f"{col}_rolling_{window}_std"] =data.groupby("air_store_id")[col].transform(lambda x: x.rolling(window).std())
        data[f"{col}_rolling_{window}_max"] =data.groupby("air_store_id")[col].transform(lambda x: x.rolling(window).max())
        data[f"{col}_rolling_{window}_min"] =data.groupby("air_store_id")[col].transform(lambda x: x.rolling(window).min())
    return data


# Find Predictions

In [84]:
submissions=[]

In [85]:
#models={}
#for i in range(5):
#    tf.keras.backend.clear_session()
#    model=Make_Model(parameters)
#    model.load_weights(f'../input/restaurants-visiters/nn_2_layers_0.001_{i}.h5')
#    models[f"model_{i}"]=model

In [86]:
def create_window(window_data):
    X={}
    X['numeric']=np.empty((1,19,len(numerical_columns)))
    for col in categorical_columns:
        X[col]=np.empty((1,19))
    assert window_data['air_store_id'].nunique()==1,"arrange data by air_store_id"
    X['numeric'][0,]=window_data.loc[:,numerical_columns].values
    for col in categorical_columns:
        X[col][0,]=window_data[col].values
    return X

In [87]:
with open("../input/restaurants-visiters/scaler_lstm_0.001_2.pkl","rb") as f:
    scaler=pickle.load(f)

In [88]:
for store_id in air_visit['air_store_id'].unique():
    train_store_data=air_visit.loc[air_visit['air_store_id']==store_id].reset_index(drop=True)
    train_store_data=train_store_data.iloc[-19:].reset_index(drop=True)
    test_store_data=test.loc[test['air_store_id']==store_id].reset_index(drop=True)
    length=test_store_data.shape[0]
    for i in tqdm(range(length)):
        train_store_data=pd.concat([train_store_data,test_store_data.iloc[i:i+1].reset_index(drop=True)],ignore_index=True)
        train_store_data=MAKE_LAG_FEATURES(train_store_data)
        train_store_data.fillna(-1,inplace=True)
        prediction_data=train_store_data.iloc[-20:-1].reset_index(drop=True)
        data=scaler.transform(prediction_data.loc[:,numerical_columns])
        data=pd.DataFrame(data,columns=scaler.feature_names_in_)
        prediction_data=prediction_data.loc[:,categorical_columns+['visit_date','visitors']].join(data)
        window_data=create_window(prediction_data)
        del data,prediction_data
        import gc
        gc.collect()
        train_store_data.loc[train_store_data.shape[0]-1,'visitors']=np.squeeze(model.predict(window_data))
    final_sub_data=train_store_data.iloc[-length:].reset_index(drop=True)
    submissions.append(final_sub_data)

100%|██████████| 39/39 [00:11<00:00,  3.34it/s]


In [89]:
submissions=pd.concat(submissions,ignore_index=True)

In [90]:
submission=submissions[['id','visitors']]

In [91]:
submission.head(5)

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,16.556656
1,air_00a91d42b08b08d9_2017-04-24,27.842381
2,air_00a91d42b08b08d9_2017-04-25,32.960217
3,air_00a91d42b08b08d9_2017-04-26,33.109539
4,air_00a91d42b08b08d9_2017-04-27,30.383799


In [92]:
submission.to_csv("submission.csv",index=False)